# 모든 train 이미지 bbox 그린 후 저장하는 코드

In [1]:
import cv2
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import os

# COCO dataset의 annotation 파일 경로와 이미지 파일 경로
"""
json 파일 경로와 datset 경로 설정
"""
ann_file = '/opt/ml/dataset/train.json'
img_dir = '/opt/ml/dataset/'

# COCO 객체 생성
coco = COCO(ann_file)

"""
bbox를 그리기 위한 color 설정
다른 color를 원할 경우 수정
"""
color_list = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (0, 255, 128), (255, 128, 0), (255, 192, 203), (64, 64, 64)]

# 카테고리 ID와 이름을 딕셔너리로 저장
cat_id_to_name = {}
for cat in coco.loadCats(coco.getCatIds()):
    cat_id_to_name[cat['id']] = cat['name']

# 이미지 ID를 순회하면서 bbox 그리기
for img_id in coco.getImgIds():
    # 이미지 파일 경로 가져오기
    img_info = coco.loadImgs(img_id)[0]
    img_name = img_info['file_name'][6:]
    img_path = img_dir + img_info['file_name']

    # 이미지 로드하기
    img = cv2.imread(img_path)

    for ann in coco.loadAnns(coco.getAnnIds(imgIds=img_id)):
        bbox = ann['bbox']
        x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        # bbox 그리기
        cv2.rectangle(img, (x, y), (x+w, y+h), color_list[ann['category_id']], 4)
        # class명 그리기
        cv2.putText(img, cat_id_to_name[ann['category_id']], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color_list[ann['category_id']], 4)

    """
    저장 경로 설정
    """
    # 저장 경로
    save_path = '/opt/ml/dataset/train_bbox5'
    # 저장 폴더 생성
    os.makedirs(save_path, exist_ok=True)
    # bbox 그린 이미지 저장
    cv2.imwrite(save_path + f'/{img_name}', img)
    print("Save Complete! path:", f"{save_path}/{img_name}")
    del img
    
    # 이미지 보여주기
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.show()
    # cv2.imshow('Image', img)
    # cv2.waitKey(0)


loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
Save Complete! path: /opt/ml/dataset/train_bbox5/0000.jpg


# 원본 이미지와 bbox 그려진 이미지 보는 코드

In [10]:
from ipywidgets import interact
import os
# 경로 합친 다음 BGR을 RGB로 바꿔서 이미지 불러오기
def get_RGB_image(data_dir, file_name):
    image_file = os.path.join(data_dir, file_name)
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# 원본 이미지 경로
origin_dir = '/opt/ml/dataset/train'
# bbox 그려진 이미지 경로
bbox_dir = '/opt/ml/dataset/train_bbox2'

# 원본 이미지 리스트
origin_list = os.listdir('/opt/ml/dataset/train')
origin_list.sort()
# bbox 그려진 이미지 리스트
bbox_list = os.listdir('/opt/ml/dataset/train_bbox2')
bbox_list.sort()

# 0부터 4882까지 슬라이드 바 생성, 총 4883장 이미지
@interact(index=(0, 4882, 1) )  
def show_samples(index=0):
    origin_image = get_RGB_image(origin_dir, origin_list[index])
    bbox_image = get_RGB_image(bbox_dir, bbox_list[index])
    
    plt.figure(figsize=(12,8))
    plt.subplot(121)
    plt.title('origin_image')
    plt.imshow(origin_image)
    
    plt.subplot(122)
    plt.title('bbox_image')
    plt.imshow(bbox_image)

interactive(children=(IntSlider(value=0, description='index', max=4882), Output()), _dom_classes=('widget-inte…